In [1]:
import phrase_entity_extraction as ph

In [2]:
sent = """2724 CARL T JONES DR SE""".lower() 

max_address = ph.return_max_address4(ph.seq, sent)
if sent.upper().strip() == max_address.upper().strip():
    print("ALL GOOD!")
else:
    print()
    print("## ************** NOT THE SAME!!!! *********************##\n")
encoding = ph.encode_from_word_list(ph.w(sent))
print("Max Address: ", max_address, "\n\nWORD ARRAY: ", ph.w(sent), "\n\nENCODING: ", encoding)
print('\n\n{},"{}"'.format(sent.upper(), encoding))


## ************** NOT THE SAME!!!! *********************##

Max Address:  2724 carl t 

WORD ARRAY:  ['2724', 'carl', 't', 'jones', 'dr', 'se'] 

ENCODING:  [['DIGIT', 'ADR_HEAD'], ['ALPHA'], ['LETTER'], ['ALPHA'], ['WAY'], ['DIR']]


2724 CARL T JONES DR SE,"[['DIGIT', 'ADR_HEAD'], ['ALPHA'], ['LETTER'], ['ALPHA'], ['WAY'], ['DIR']]"


In [3]:
ph.seq.look_ahead(ph.encode_from_word_list(ph.w(sent.lower()))).get_active_sequences()

[]

In [4]:
ph.seq.look_ahead(ph.encode_from_word_list(ph.w(sent.lower()))).get_next_values()

[]

In [5]:
markers = ph.get_markers(ph.seq, sent, ['_ADDRESS_', '_POBOX_', '_SUITE_', '_DIR_'])
markers

[[0, 1, 1, ['_SUITE_'], '2724'],
 [0, 2, 2, ['_ADDRESS_'], '2724 carl'],
 [0, 5, 5, ['_ADDRESS_'], '2724 carl t jones dr'],
 [2, 3, 1, ['_SUITE_'], 't'],
 [5, 6, 1, ['_DIR_'], 'se']]

In [6]:
class BreathFirstSearch:
    def __init__(self, markers):
        self.markers = markers
    def end(self, node):
        return node[1]
    def start(self, node):
        return node[0]
    def type(self, node):
        return node[3][0]
    def length(self, node):
        return node[2]
    def rep(self, node):
        return node[4]
    def get_successors(self, current_node):
        next_matches = [n 
            for n in self.markers 
            if self.start(n) == self.end(current_node) and self.type(n) != self.type(current_node) and self.type(n) != '_POBOX_']
        return next_matches[:]
    def get_branches(self, node):
        fringe = [[node[:]]]
        branches = []
        while fringe:
            activeNode = fringe.pop()
            successors = self.get_successors(activeNode[-1])               
            if successors:
                for successor in successors:
                    nextNode = activeNode[:]
                    nextNode.append(successor)
                    fringe.append(nextNode[:])
            else:
                branches.append(activeNode)
        return branches
    def get_all_branches(self):
        all_branches = []
        for node in self.markers:
            for branch in self.get_branches(node):
                all_branches.append(branch)
        return all_branches
        
bfs = BreathFirstSearch(markers)

#print("SUCCESSORS: ", bfs.get_successors([0, 2, 2, ['_SUITE_'], 'apt 10']))
print("BRANCH:")
for branch in bfs.get_branches([0, 2, 2, ['_SUITE_'], 'apt 10']):
    print(branch)
print("\nALL BRANCHES:")
for branch in bfs.get_all_branches():
    print(branch)
        

BRANCH:
[[0, 2, 2, ['_SUITE_'], 'apt 10']]

ALL BRANCHES:
[[0, 1, 1, ['_SUITE_'], '2724']]
[[0, 2, 2, ['_ADDRESS_'], '2724 carl'], [2, 3, 1, ['_SUITE_'], 't']]
[[0, 5, 5, ['_ADDRESS_'], '2724 carl t jones dr'], [5, 6, 1, ['_DIR_'], 'se']]
[[2, 3, 1, ['_SUITE_'], 't']]
[[5, 6, 1, ['_DIR_'], 'se']]


In [10]:
seq2 = ph.Hydraseq('second')
seq2.insert("_SUITE_ _ADDRESS_ *KEEP*")
seq2.insert("_ADDRESS_ *KEEP*")
seq2.insert("_ADDRESS_ _SUITE_ *KEEP*")
seq2.insert("_ADDRESS_ _DIR_ *KEEP*")
seq2.insert("_ADDRESS_ _DIR_ _SUITE_ *KEEP*")
seq2.insert("_POBOX_ *KEEP*")
keepers = [branch for branch in bfs.get_all_branches() if 'KEEP' in seq2.look_ahead( [node[3] for node in branch] ).get_next_values()]
print(keepers)
max_len = 0
max_branch = None
for branch in keepers:
    len_branch = branch[-1][1] - branch[0][0]
    if len_branch > max_len:
        max_len = len_branch
        max_branch = branch
    
print(max_branch)

for branch in bfs.get_all_branches():
    print("branch:" ,branch)
    arr = [node[3] for node in branch]
    print("array: ", arr)
    thesequence = seq2.look_ahead(arr).get_next_values()
    print("ISITAKEEPER?=====\t", thesequence)
    
    
print("THIS SHULD BE KEPPER: ", seq2.look_ahead([['_ADDRESS_'],['_SUITE_','_DIR_']]).get_next_values())

[[[0, 2, 2, ['_ADDRESS_'], '2724 carl'], [2, 3, 1, ['_SUITE_'], 't']], [[0, 5, 5, ['_ADDRESS_'], '2724 carl t jones dr'], [5, 6, 1, ['_DIR_'], 'se']]]
[[0, 5, 5, ['_ADDRESS_'], '2724 carl t jones dr'], [5, 6, 1, ['_DIR_'], 'se']]
branch: [[0, 1, 1, ['_SUITE_'], '2724']]
array:  [['_SUITE_']]
ISITAKEEPER?=====	 ['_ADDRESS_']
branch: [[0, 2, 2, ['_ADDRESS_'], '2724 carl'], [2, 3, 1, ['_SUITE_'], 't']]
array:  [['_ADDRESS_'], ['_SUITE_']]
ISITAKEEPER?=====	 ['KEEP']
branch: [[0, 5, 5, ['_ADDRESS_'], '2724 carl t jones dr'], [5, 6, 1, ['_DIR_'], 'se']]
array:  [['_ADDRESS_'], ['_DIR_']]
ISITAKEEPER?=====	 ['KEEP', '_SUITE_']
branch: [[2, 3, 1, ['_SUITE_'], 't']]
array:  [['_SUITE_']]
ISITAKEEPER?=====	 ['_ADDRESS_']
branch: [[5, 6, 1, ['_DIR_'], 'se']]
array:  [['_DIR_']]
ISITAKEEPER?=====	 []
THIS SHULD BE KEPPER:  ['KEEP', '_SUITE_']


In [11]:
nodes =  [[0, 5, 5, ['_ADDRESS_'], '11500 s sam houston parkway'], [5, 6, 1, ['_SUITE_', '_DIR_'], 'w']]
[node[3] for node in nodes]

[['_ADDRESS_'], ['_SUITE_', '_DIR_']]

In [12]:
### DOUBLE DECKER APPROACH
def return_max_address4(seq, sent):
    markers = ph.get_markers(seq, sent, ['_ADDRESS_', '_POBOX_', '_SUITE_', '_DIR_'])
    class BreathFirstSearch:
        def __init__(self, markers):
            self.markers = markers
        def end(self, node):
            return node[1]
        def start(self, node):
            return node[0]
        def type(self, node):
            return node[3][0]
        def length(self, node):
            return node[2]
        def rep(self, node):
            return node[4]
        def get_successors(self, current_node):
            next_matches = [n for n in self.markers if self.start(n) == self.end(current_node) and self.type(n) != self.type(current_node) and self.type(n) != '_POBOX_']
            return next_matches[:]
        def get_branches(self, node):
            fringe = [[node[:]]]
            branches = []
            while fringe:
                activeNode = fringe.pop()
                successors = self.get_successors(activeNode[-1])
                if successors:
                    for successor in successors:
                        nextNode = activeNode[:]
                        nextNode.append(successor)
                        fringe.append(nextNode[:])
                else:
                    branches.append(activeNode)
            return branches
        def get_all_branches(self):
            all_branches = []
            for node in self.markers:
                for branch in self.get_branches(node):
                    all_branches.append(branch)
            return all_branches

    bfs = BreathFirstSearch(markers)
    seq2 = ph.Hydraseq('second')
    seq2.insert("_SUITE_ _ADDRESS_ *KEEP*")
    seq2.insert("_ADDRESS_ *KEEP*")
    seq2.insert("_ADDRESS_ _SUITE_ *KEEP*")
    seq2.insert("_ADDRESS_ _DIR_ _SUITE_ *KEEP*")
    seq2.insert("_POBOX_ *KEEP*")
    keepers = [branch for branch in bfs.get_all_branches() if 'KEEP' in seq2.look_ahead([node[3] for node in nodes] ).get_next_values()]

    max_len = 0
    max_branch = None
    for branch in keepers:
        len_branch = branch[-1][1] - branch[0][0]
        if len_branch > max_len:
            max_len = len_branch
            max_branch = branch

    #print("HERE: ",max_branch)
    return " ".join([item[4] for item in max_branch])

return_max_address4(ph.seq, sent)

'2724 carl t jones dr se'

In [ ]:
filtered_markers = [marker for marker in markers if not (marker[0] == 0 and marker[2] == 1)]
print(filtered_markers)


In [ ]:
def entitys_overlap(ent1, ent2):
    if ent1[1] <= ent2[0] or ent2[1] <= ent1[0]:
        return False
    else:
        return True

def get_sorted_entity(_markers, entity):
    entities = [arr for arr in _markers if arr[3][0] == entity]
    entities.sort(key=lambda x: int(x[2]))
    return entities
def get_all_sorted_entity(_markers, axis):
    """axis 0:left, 1:right, 2:length, 3:lst<type>, 4:str"""
    markers.sort(key=lambda x: int(x[axis]))
    return markers

In [ ]:
def get_markers(seq, sent, lst_targets):
    """Input is like '123 main str' and returns a list of lists
        RETURNS: a list of list, each list being a candidate and having these values.
            idx_beg, idx_end, length, matches (ADDRESS etc), sequence
        ATTN!!  this lowercases stuff, TODO: Generalize this so it doesn't need lowercasing
    """
    sent = str(sent).lower().strip()
    arr_w = ph.w(sent)
    idx_tail = len(arr_w)
    markers = []

    for idx_beg in range(idx_tail):
        for idx_end in range(idx_beg + 1, idx_tail +1):
            next_values = seq.look_ahead(ph.encode_from_word_list(arr_w[idx_beg:idx_end])).get_next_values()
            matches = list(set(next_values) & set(lst_targets) )
            if matches:
                markers.append([idx_beg, idx_end, idx_end - idx_beg, matches, ' '.join(arr_w[idx_beg:idx_end])])

    return markers


st = sent
markers = get_markers(ph.seq, st, ['_ADDRESS_', '_POBOX_', '_SUITE_', '_DIR_'])
markers

In [ ]:
def sift_markers(markers):
    boxes = [marker for marker in markers if marker[3][0] == '_POBOX_']
    addresses = [marker for marker in markers if marker[3][0] == '_ADDRESS_']
    suites = [marker for marker in markers if marker[3][0] == '_SUITE_']
    dirs = [marker for marker in markers if marker[3][0] == '_DIR_']
    new_markers = boxes[:]
#     for address in addresses:
#         if not any([entitys_overlap(address, item) for item in new_markers]):
#             new_markers.append(address)
    new_markers.extend(addresses)
    for suite in suites:
        if not any([entitys_overlap(suite, item) for item in new_markers]):
            new_markers.append(suite)
    for diritem in dirs:
        if not any([entitys_overlap(diritem, item) for item in new_markers]):
            new_markers.append(diritem)
    return new_markers

sift_markers(markers)

In [ ]:


def get_best_fit_4(seq, st, lst_entities):
    assert isinstance(lst_entities,list), "expects lst_entities is [ str, str ]"
    assert isinstance(lst_entities[0], str), "expects lst_entities is [ str, str ]"
    assert isinstance(st, str), "expects st to be a str"
    def sift_markers(markers):
        boxes = [marker for marker in markers if marker[3][0] == '_POBOX_']
        addresses = [marker for marker in markers if marker[3][0] == '_ADDRESS_']
        suites = [marker for marker in markers if marker[3][0] == '_SUITE_']
        dirs = [marker for marker in markers if marker[3][0] == '_DIR_']
        new_markers = boxes[:]
#         for address in addresses:
#             if not any([entitys_overlap(address, item) for item in new_markers]):
#                 new_markers.append(address)
        new_markers.extend(addresses)
        for suite in suites:
            if not any([entitys_overlap(suite, item) for item in new_markers]):
                new_markers.append(suite)
        for diritem in dirs:
            if not any([entitys_overlap(diritem, item) for item in new_markers]):
                new_markers.append(diritem)
        return new_markers
    def get_sorted_indexes(markers, which, up=True):
        sorted_list = sorted(list(set([marker[which] for marker in markers])))
        return sorted_list if up else sorted_list[::-1]
    
    
    def book_best_fit(arr_domain, markers):
        def is_in_dictionary(markers, start, endplus):
            match_starts = [item for item in markers if item[0] == start]
            match_both = [item for item in match_starts if item[1] == endplus]
            return match_both
        
        last_length = [-1]*len(arr_domain)
        for i in range(len(arr_domain)):
            if is_in_dictionary(markers, 0, i+1):
                last_length[i] = i + 1
            if last_length[i] == -1:
                for j in range(i):
                    if last_length[j] != -1 and is_in_dictionary(markers, j+1, i+1):
                        last_length[i] = i - j
                        break
        print("last_length: ", last_length)                
        decompositions = []
        if last_length[-1] != -1:
            idx = len(arr_domain) - 1
            while idx >= 0:
                decompositions.append(is_in_dictionary(markers,idx + 1 - last_length[idx], idx + 1)[0])
                idx -= last_length[idx]
            decompositions = decompositions[::-1]
        return decompositions
    
    
    markers = get_markers(ph.seq, st, lst_entities)
    markers = sift_markers(markers)
#     print("MARKERS TO WORK WITH: ", markers)
    lefts = get_sorted_indexes(markers, 0, up=True)
    rights = get_sorted_indexes(markers, 1, up=False)
    print(lefts)
    print(rights)
    final_decomps = []
    for left in lefts:
        for right in rights:
            subject = ph.w(st)[left:right]
            if not subject or (right - left) <=1:
                continue
            markers = get_markers(ph.seq, " ".join(subject), lst_entities)
            markers = sift_markers(markers)
            decomp = book_best_fit(subject, markers)
            if decomp:
                print("decomp: ", left, right, subject,'\n',decomp, '\n')
                final_decomps.append(decomp)
    #return final_decomps

    best_decomp = None
    max_len = 0
    for decomp in final_decomps:  
        if decomp[-1][1] > max_len:
            max_len = decomp[-1][1]
            best_decomp = decomp
    
    return best_decomp if best_decomp else []
    

def return_max_address4(seq, st):
    lst_final_ents = [['_ADDRESS_', '_SUITE_', '_DIR_'], ['_POBOX_']]
    for lst_entities in lst_final_ents:
        best_fit = get_best_fit_4(seq, st, lst_entities)
        #print("Best_fit: ", best_fit, lst_entities[0])
        if any([fit[3][0] == lst_entities[0] for fit in best_fit]):
            str_rep = " ".join([fit[4] for fit in best_fit])
            return str_rep
    else:
        return []
    
final_decomps = get_best_fit_4(ph.seq, st, ['_ADDRESS_',  '_SUITE_', '_DIR_'])
[print("DECOMP: ", decomp) for decomp in final_decomps]

final = return_max_address4(ph.seq, st)

print("\nAND THE BEST IS\n", final)

apt 10 2700 webers lane

drops the apt, because we put in the large address in first wich blocks teh apt 10.

maybe we need to iterate over best fits and checking if address made it, if not then get rid of the first 0-x try and try again with next?